In [1]:
pip install torch transformers scikit-learn pandas numpy gensim sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
from collections import Counter
device = torch.device("cpu")


### Load CSV file


In [3]:
# =========================
# 1. Load datasets
# =========================
df1 = pd.read_csv("humor_v1_burmese - classification.csv")  # label, text_data
df2 = pd.read_csv("humor_v1_burmese_pl_extract.csv")  # title, humor_data, punchline

# =========================
# 2. Merge and label
# =========================
df2["text_data"] = (
    df2["title"].fillna('') + " " +
    df2["humor_data"].fillna('') + " " +
    df2["punchline"].fillna('')
)
df2["label"] = 1  # assign humor label (adjust if needed)

# Keep necessary cols
df1 = df1[["label", "text_data"]]
df2 = df2[["label", "text_data"]]

# Combine
df = pd.concat([df1, df2], ignore_index=True)

# =========================
# 3. Clean and encode labels
# =========================
df["text_data"] = df["text_data"].astype(str).str.strip()
df = df[df["text_data"].astype(bool)]
df["label"] = df["label"].fillna("UNKNOWN").astype(str)

label_map = {label: idx for idx, label in enumerate(sorted(df["label"].unique()))}
labels = [label_map[l] for l in df["label"]]
texts = df["text_data"].tolist()

# =========================
# 4. Build vocab & encode texts
# =========================
def build_vocab(texts, min_freq=1):
    counter = Counter()
    for text in texts:
        counter.update(text.split())
    vocab = {"<PAD>": 0, "<UNK>": 1}
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = len(vocab)
    return vocab

vocab = build_vocab(texts)
vocab_size = len(vocab)

def encode_text(text, vocab):
    return [vocab.get(word, vocab["<UNK>"]) for word in text.split()]

encoded_texts = [torch.tensor(encode_text(t, vocab)) for t in texts]

### Dataset class

In [4]:
class HumorDataset(Dataset):
    def __init__(self, enc_texts, labels):
        self.enc_texts = enc_texts
        self.labels = labels
    def __len__(self):
        return len(self.enc_texts)
    def __getitem__(self, idx):
        return self.enc_texts[idx], self.labels[idx]

def collate_fn(batch):
    texts, labels = zip(*batch)
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0)
    return padded_texts, torch.tensor(labels, dtype=torch.long)

### Embeddings functions

#### Random embeddings

In [5]:
def random_embeddings(texts, emb_dim=100):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    X_counts = vectorizer.fit_transform(texts)
    vocab_size = len(vectorizer.vocabulary_)
    embedding_matrix = np.random.rand(vocab_size, emb_dim)  
    X_emb = X_counts @ embedding_matrix  
    return X_emb  


#### fastText embeddings

In [6]:
def fasttext_embeddings(texts, fasttext_path="cc.my.300.bin"):
    ft = KeyedVectors.load_word2vec_format(fasttext_path, binary=True)
    def sent_vec(sentence):
        words = sentence.split()
        vecs = [ft[w] for w in words if w in ft]
        return np.mean(vecs, axis=0) if vecs else np.zeros(ft.vector_size)
    return np.array([sent_vec(t) for t in texts])


#### mBERT embeddings

In [7]:
device = torch.device("cpu")
def mbert_embeddings(texts, model_name="bert-base-multilingual-cased"):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    model.eval()
    embeddings = []
    for t in texts:
        inputs = tokenizer(t, return_tensors='pt', truncation=True, padding=True, max_length=50)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return np.array(embeddings)

#### multilingual e5 embeddings

In [8]:
def e5_embeddings(texts, model_name="intfloat/multilingual-e5-small"):
    model = SentenceTransformer(model_name)
    return model.encode(texts, convert_to_numpy=True)

### BiLSTM Model

In [9]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout, num_classes):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers,
                            dropout=dropout if num_layers > 1 else 0,
                            bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.dropout(x[:, -1, :])
        return self.fc(x)

### Train and Evaluation functions

In [10]:
def train_model(model, train_loader, val_loader, epochs, lr, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds.extend(outputs.argmax(1).cpu().numpy())
            true_labels.extend(y_batch.numpy())
    return accuracy_score(true_labels, preds), classification_report(true_labels, preds, digits=4, output_dict=True)
        

### GridSearch & embeddings

In [11]:
param_grid = {
    "embed_dim": [100, 200],
    "hidden_dim": [64, 128],
    "num_layers": [1, 2],
    "dropout": [0.3, 0.5],
    "lr": [0.001, 0.0005]
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_summary = []

for embed_dim in param_grid["embed_dim"]:
    for hidden_dim in param_grid["hidden_dim"]:
        for num_layers in param_grid["num_layers"]:
            for dropout in param_grid["dropout"]:
                for lr in param_grid["lr"]:
                    fold_accs = []
                    for train_idx, val_idx in kf.split(encoded_texts):
                        X_train = [encoded_texts[i] for i in train_idx]
                        y_train = [labels[i] for i in train_idx]
                        X_val = [encoded_texts[i] for i in val_idx]
                        y_val = [labels[i] for i in val_idx]
                        train_loader = DataLoader(HumorDataset(X_train, y_train), batch_size=32, collate_fn=collate_fn, shuffle=True)
                        val_loader = DataLoader(HumorDataset(X_val, y_val), batch_size=32, collate_fn=collate_fn)
                        model = BiLSTM(vocab_size, embed_dim, hidden_dim, num_layers, dropout, num_classes=len(label_map))
                        acc, _ = train_model(model, train_loader, val_loader, epochs=5, lr=lr, device=device)
                        fold_accs.append(acc)
                    results_summary.append({
                        "embed_dim": embed_dim,
                        "hidden_dim": hidden_dim,
                        "num_layers": num_layers,
                        "dropout": dropout,
                        "lr": lr,
                        "mean_acc": np.mean(fold_accs),
                        "std_acc": np.std(fold_accs)
                    })

best_result = max(results_summary, key=lambda x: x["mean_acc"])
print("\nBest Hyperparameters:", best_result)
print(f"Accuracy: {best_result['mean_acc']:.4f} ± {best_result['std_acc']:.4f}")

/Users/zinmarmyint/anaconda3/envs/pyqt_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/zinmarmyint/anaconda3/envs/pyqt_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/zinmarmyint/anaconda3/envs/pyqt_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

KeyboardInterrupt: 